In [54]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pandas
from sklearn import datasets
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.manifold import MDS
from sklearn.preprocessing import StandardScaler

In [55]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(axis=1)
    return df[indices_to_keep].astype(np.float64)

In [56]:
client_df = pd.read_csv('Clientes_descripcion.txt',delimiter='|',header=None)
movies_df = pd.read_csv('Peliculas_descripcion.txt', delimiter='|', header=None)
rating_df = pd.read_csv('Peliculas_Clientes.txt', delim_whitespace=True, header=None)

client_df = client_df.drop(client_df.iloc[:,4:5], axis=1)
movies_df = movies_df.drop(movies_df.iloc[:,3:5], axis=1)
movies_df = movies_df.drop(movies_df.iloc[:,3:], axis=1)
rating_df = rating_df.drop(rating_df.iloc[:,3:4], axis=1)

client_columns = ['id','age','sex','profession']
client_df.columns = client_columns

movies_columns = ['id','name','release_date']
movies_df.columns = movies_columns

rating_columns = ['user_id','movie_id','rating']
rating_df.columns = rating_columns

client_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 943 entries, 0 to 942
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          943 non-null    int64 
 1   age         943 non-null    int64 
 2   sex         943 non-null    object
 3   profession  943 non-null    object
dtypes: int64(2), object(2)
memory usage: 29.6+ KB


In [57]:
rating_matrix = rating_df.pivot_table(index='movie_id', columns='user_id', values='rating')

rating_matrix.index = movies_df.set_index('id')['name']
rating_matrix


user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
name,,,,,,,,,,,,,,,,,,,,,
Toy Story (1995),5.0,4.0,NaN,NaN,4.0,4.0,NaN,NaN,NaN,4.0,...,2.0,3.0,4.0,NaN,4.0,NaN,NaN,5.0,NaN,NaN
GoldenEye (1995),3.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
Four Rooms (1995),4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Get Shorty (1995),3.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,4.0,...,5.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN
Copycat (1995),3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Mat' i syn (1997),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B. Monkey (1998),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sliding Doors (1998),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
evalMovie = rating_matrix.apply(lambda x: x.count(), axis=1)
evalMovie = evalMovie.sort_values(ascending=False)
evalMovie

name
Star Wars (1977)                                      583
Contact (1997)                                        509
Fargo (1996)                                          508
Return of the Jedi (1983)                             507
Liar Liar (1997)                                      485
                                                     ... 
Hungarian Fairy Tale, A (1987)                          1
Death in the Garden (Mort en ce jardin, La) (1956)      1
Every Other Weekend (1990)                              1
Baton Rouge (1988)                                      1
Scream of Stone (Schrei aus Stein) (1991)               1
Length: 1682, dtype: int64

In [59]:
evalUser = rating_matrix.apply(lambda x: x.count(), axis=0)
evalUser = evalUser.sort_values(ascending=False)
evalUser

user_id
405    737
655    685
13     636
450    540
276    518
      ... 
685     20
475     20
36      20
732     20
596     20
Length: 943, dtype: int64

In [60]:
#1. Filtrar las películas y usuarios para quedarnos sólo con:
#50 películas (las 50 que tengan mayor número de evaluaciones)


scaler = StandardScaler()
scaler.fit_transform(rating_matrix)

pca = PCA()
pca.fit_transform(rating_matrix)

pca.n_components_ =  50


#100 usuarios (los 100 que hayan realizado el mayor número de evaluaciones)





ValueError: Input contains NaN, infinity or a value too large for dtype('float64').